In [185]:
import pandas as pd
import numpy as np

In [256]:
data = pd.read_csv('data/heart.csv', skiprows=[1, 2], header=0)
D = data.drop('HeartDisease', axis=1)
C = data['HeartDisease']

In [193]:
def dummify(df):
    df_dummy = df.copy()
    cat_cols = [col for col in df.columns if pd.api.types.is_object_dtype(df[col])]

    for col in cat_cols:
        cats = set(df[col])
        for cat in cats:
            df_dummy[col + '_' + str(cat)] = (df[col] == cat) * 1
        df_dummy.drop(col, axis=1, inplace=True)

    return df_dummy

In [190]:
def standardize(df):
    df_std = df.copy()
    num_cols = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]

    for col in num_cols:
        df_std[col] = (df_std[col] - df_std[col].mean()) / df_std[col].std()

    return df_std

In [188]:
def dist(x1, x2):
    return np.sqrt(((x1 - x2)**2).sum())

In [251]:
def knn(row, D, C, k):
    top_k_dists = D.apply(lambda x: dist(x, row), axis=1).sort_values().head(k)
    top_k_c = C.loc[top_k_dists.index]

    counts = top_k_c.value_counts()
    if len(counts[counts == counts.max()]) > 1:
        return top_k_c.iloc[0]
    else:
        return counts.idxmax()

In [252]:
knn(D.loc[0], D.drop(0), C.drop(0), 5)

0

In [263]:
def knn_predict(D, k, classvar = None):
    D_orig = D.copy()[[]]
    C = D_orig[classvar]
    D = dummify(standardize(D.drop(classvar, axis=1)))
    D_orig['pred'] = pd.Series(D.index).apply(lambda row: knn(D.loc[row], D.drop(row), C.drop(row), k))
    return D_orig

In [265]:
results = knn(data, 10, 'HeartDisease')
results

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,pred
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1,0
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1,0


In [266]:
pd.crosstab(results['HeartDisease'], results['pred'])

pred,0,1
HeartDisease,,
0,338,72
1,51,457
